In [1]:
import re
import json
import os

In [25]:
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
from legal_document_splitter import *

In [4]:
DATA_PATH="data/"

In [5]:
json_normas_path = DATA_PATH + 'normas_full.json'

with open(json_normas_path, 'r') as file:
    normas_json = json.load(file)

normas = list(normas_json.values())

In [6]:
normas_json.keys()

dict_keys(['Ato Declaratório Cosar nº 47.txt', 'Ato Declaratório Interpretativo RFB nº 1.txt', 'Ato Declaratório Interpretativo SRF nº 2.txt', 'Ato Declaratório Interpretativo SRF nº 24.txt', 'Ato Declaratório Interpretativo SRF nº 5.txt', 'Ato Declaratório PGFN nº 1.txt', 'Ato Declaratório PGFN nº 13.txt', 'Ato Declaratório PGFN nº 14.txt', 'Ato Declaratório PGFN nº 2.txt', 'Ato Declaratório PGFN nº 3.txt', 'Ato Declaratório PGFN nº 4.txt', 'Ato Declaratório PGFN nº 5.txt', 'Ato Declaratório PGFN nº 6 DE 2006.txt', 'Ato Declaratório PGFN nº 6 DE 2008.txt', 'Ato Declaratório PGFN nº 9.txt', 'Ato Declaratório RFB nº 16.txt', 'Ato Declaratório RFB nº 18.txt', 'Ato Declaratório RFB nº 3.txt', 'Ato Declaratório SRF nº 14.txt', 'Ato Declaratório SRF nº 16.txt', 'Ato Declaratório SRF nº 22.txt', 'Ato Declaratório SRF nº 26.txt', 'Ato Declaratório SRF nº 28.txt', 'Ato Declaratório SRF nº 3.txt', 'Ato Declaratório SRF nº 48.txt', 'Ato Declaratório SRF nº 7.txt', 'Ato Declaratório SRF nº 8.txt'

In [7]:
hierarchical_text_splitter = HierachicalRecursiveCharacterTextSplitter(
    separators=LEGISLATION_SPLITTING_HIERARCHY_BRAZIL,
    is_separator_regex=True,
    chunk_size=1000,  # Tamanho máximo do chunk em caracteres
    chunk_overlap=100,  # Sobreposição entre chunks para contexto
    apply_chunk_size=6 # Separator index to start applying the chunk size check.
)

In [18]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Tamanho máximo do chunk em caracteres
    chunk_overlap=100,  # Sobreposição entre chunks para contexto
)

In [15]:
tokens_counter = tokensCounter("stjiris/bert-large-portuguese-cased-legal-tsdae")

### Process (segmentation and flattening) all the referenced documents

In [16]:
from tqdm import tqdm

In [29]:
applied_splitter = []
passages_referred_docs = []

for filename, doc in tqdm(normas_json.items(), desc="Processing all referred legal documents"):
    # print(f"Processing file: {filename}")
    
    # Remove a extensão ".txt" do nome do arquivo
    doc_name = filename.replace(".txt", "")

    # First try applying the hierarchical splitter
    chunks = hierarchical_text_splitter.split_text(doc)
    passages = []
    current_path = ""

    flatten_passages_hierarchy(chunks, current_path, passages)
    
    # print(chunks.keys())
    
    total_statistics = tokens_counter.passages_statistics([passage['passage'] for passage in passages])

    # print(total_statistics)

    #
    # Check if there are indications the hierarchical splitter did not work
    # verifying if there is any chunk longer than the maximum size.
    #
    
    if total_statistics['max_tokens'] > 512:
        # For such document, apply a regular text splitter.
        
        print(f">>> Applying regular text splitter in file {filename}.")

        chunks = text_splitter.split_text(doc)
        applied_splitter.append({'filename': filename,
                                 'splitter': 'text'})

        for passage in chunks:
            # Formata cada chunk com o nome do arquivo e o conteúdo
            passage_with_path = "{}: {}".format(doc_name, passage)
            passages_referred_docs.append(passage_with_path)    
    else:
        applied_splitter.append({'filename': filename,
                                 'splitter': 'hierarchical'})
    
        for passage in passages:
            # Formata cada chunk com o nome do arquivo e o conteúdo
            passage_with_path = "{}: {}".format(doc_name + "_" + passage['path'], passage['passage'])
            passages_referred_docs.append(passage_with_path)

Processing all referred legal documents:   0%|                                                                                                                          | 0/332 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1087: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1087: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
Processing all referred legal documents:   5%|██████                                                                                                          | 18/332 [00:00<00:01, 159.66it/s]

>>> Applying regular text splitter in file Ato Declaratório PGFN nº 1.txt.
>>> Applying regular text splitter in file Ato Declaratório PGFN nº 6 DE 2006.txt.
>>> Applying regular text splitter in file Ato Declaratório PGFN nº 9.txt.
>>> Applying regular text splitter in file Ato Declaratório RFB nº 3.txt.
>>> Applying regular text splitter in file Decreto nº 21.177.txt.


Processing all referred legal documents:  13%|██████████████▎                                                                                                  | 42/332 [00:01<00:08, 35.19it/s]

>>> Applying regular text splitter in file Decreto nº 3.000.txt.
>>> Applying regular text splitter in file Decreto nº 5.128.txt.
>>> Applying regular text splitter in file Decreto nº 50.656.txt.
>>> Applying regular text splitter in file Decreto nº 52.288.txt.
>>> Applying regular text splitter in file Decreto nº 56.435.txt.
>>> Applying regular text splitter in file Decreto nº 57.784.txt.
>>> Applying regular text splitter in file Decreto nº 57.942.txt.
>>> Applying regular text splitter in file Decreto nº 59.308.txt.
>>> Applying regular text splitter in file Decreto nº 59.309.txt.


Processing all referred legal documents:  15%|████████████████▋                                                                                                | 49/332 [00:01<00:07, 35.87it/s]

...

>>> Applying regular text splitter in file RIR2018.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 100.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 105.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 115.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 123.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 134.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 135.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 138.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 14.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 140.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 144.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 147.txt.
>>> Applying regular text splitter in file Solução de 

Processing all referred legal documents:  93%|████████████████████████████████████████████████████████████████████████████████████████████████████████▏       | 309/332 [00:14<00:00, 64.16it/s]

>>> Applying regular text splitter in file Solução de Consulta Cosit nº 206.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 209.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 211.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 214.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 240.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 256.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 313.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 354.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 356.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 48.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 6.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 60.txt.
>>> Applying regular text splitt

Processing all referred legal documents: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 332/332 [00:14<00:00, 22.42it/s]

>>> Applying regular text splitter in file Solução de Consulta Cosit nº 79.txt.
>>> Applying regular text splitter in file Solução de Consulta Cosit nº 82.txt.


In [30]:
splitter_df = pd.DataFrame(applied_splitter)

In [31]:
splitter_df.groupby("splitter").count()

,filename
splitter,
hierarchical,171
text,161


In [32]:
len(passages_referred_docs)

20464

In [28]:
splitter_df.groupby("splitter").count()

,filename
splitter,
hierarchical,250
text,82


In [23]:
len(passages_referred_docs)

24281

#### Check data statistics

In [24]:
tokens_counter.passages_statistics(passages_referred_docs)

{'max_tokens': 1007,
 'min_tokens': 18,
 'mean_tokens': 165.15407108438697,
 'std_tokens': 105.98080524276165,
 'skewness_tokens': 1.5930594614675477,
 'kurtosis_tokens': 6.1454833961628985,
 'median_tokens': 160,
 'max_token_passage': 3220}

In [21]:
tokens_counter.passages_statistics(passages_referred_docs)

{'max_tokens': 966,
 'min_tokens': 19,
 'mean_tokens': 202.92464816262705,
 'std_tokens': 94.56423146322919,
 'skewness_tokens': 0.8993632242460677,
 'kurtosis_tokens': 5.14506011498575,
 'median_tokens': 218.0,
 'max_token_passage': 11027}

In [33]:
import pickle

In [34]:
with open(DATA_PATH + "chunks_normas_new_20250228.pkl", "wb") as output_file:
    pickle.dump(passages_referred_docs, output_file, pickle.HIGHEST_PROTOCOL)